<a href="https://colab.research.google.com/github/iraidaantropova/StrD/blob/main/HW8__SinitsaI_StrD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("lesson8")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
#считывание данных
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)
df.show(10)


+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [ ]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [ ]:
#для дальнейшей работы данные дб в числовом формате, в Cruise_line используется строковый,
#с помощью StringIndexer тип данных будет преобразован
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)

for item in indexed.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#создание векторов из объектов, тк данные дб в векторной форме

assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
output=assembler.transform(indexed)
output.select('features','crew').show(5)


+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [ ]:
final_data=output.select('features','crew')
#разделение данных на  train и test
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               111|
|   mean| 7.832702702702711|
| stddev|3.6117260909799547|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                47|
|   mean| 7.703191489361703|
| stddev|3.2691829230365514|
|    min|              2.95|
|    max|              19.1|
+-------+------------------+



In [ ]:
from pyspark.ml.regression import LinearRegression
#создание объекта класса линейной регрессии
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')

trained_ship_model=ship_lr.fit(train_data)

ship_results=trained_ship_model.evaluate(train_data)

print('Rsquared Error :',ship_results.r2)


Rsquared Error : 0.9491555658909803


точность модели составляет 91%

In [ ]:
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[6.0,90.0,20.0,9....|
|[6.0,113.0,37.82,...|
|[6.0,158.0,43.7,1...|
|[7.0,158.0,43.7,1...|
+--------------------+
only showing top 5 rows



In [ ]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()


+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.287524451864167|
|[6.0,90.0,20.0,9....|10.225503302062517|
|[6.0,113.0,37.82,...|11.349543059689946|
|[6.0,158.0,43.7,1...|13.769582521371916|
|[7.0,158.0,43.7,1...|13.691529273748978|
|[9.0,90.09,25.01,...| 9.246310281601565|
|[9.0,110.0,29.74,...| 11.88504599910579|
|[9.0,113.0,26.74,...|11.242970206574059|
|[10.0,91.62700000...|  9.30666365413853|
|[11.0,85.0,18.48,...| 8.896538383046467|
|[11.0,86.0,21.24,...| 9.540670806419902|
|[11.0,90.09,25.01...| 8.884073698497176|
|[11.0,110.0,29.74...|11.880475934390113|
|[12.0,42.0,14.8,7...| 6.595068603317582|
|[12.0,58.6,15.66,...| 7.454182382019432|
|[12.0,88.5,21.24,...| 9.514254327458199|
|[12.0,138.0,31.14...| 12.85798590515905|
|[13.0,25.0,3.82,5...|3.0269530158713756|
|[13.0,101.509,27....|10.900905962913132|
|[14.0,30.27699999...| 3.368354615296392|
+--------------------+------------

In [ ]:
try:
    ship_lr.write().overwrite().save('ship_lr_Model')
    print('сохранение успешно')
except:
    print('ошибка')

сохранение успешно


In [ ]:
spark.stop()